# Analisi dataset tranding di Youtube


Segue l'analisi dei dati di trending di youtube pendendo spunto da questo dataset: https://www.kaggle.com/datasnaek/youtube-new e studiando il codice del suo creatore: https://github.com/mitchelljy/Trending-YouTube-Scraper  
questo tizio mi sembra ci abbia fatto sopra un bel lavoro: https://www.kaggle.com/donyoe/exploring-youtube-trending-statistics-eda  <br>
le V che possiamo sfruttare sono: 
- __Velocity__ (perchè sono dati presi giornalmente quindi real time)
- __Variety__ (ci sono anche dei file json e poi si potrebbe integrare con altri dati) 
- volendo si può pensare anche a __Volume__ il dataset su kaggle è mezzo giga e sono 6 mesi di rilevazioni su una decina di paesi (ma la vedo complessa)

## NOTA BENE

- Non si può (almeno credo) prendere dati di trending di altri giorni se non lo stesso giorno che si fa lo scraping
- Manca la fusione con i json per ottenere la categoria!!!
- Manca la popolazione dei paesi

## Analisi tranding UK

### Punti tenuti in considerazione

- Video in trending più _vecchio_ pubblicato
- Video con _visualizzazioni likes dislikes commenti_ maggiori
- Media di _visualizzazioni likes dislikes commenti_ del trending delle giornata
- _Categoria_ che riscuote maggior successo (quindi maggior numero di video in quella categoria e confronto con gli altri paesi)
- Popolazione dei paesi rispetto ai vari indicatori
- Analizzare in base ai gruppi di età

In [1]:
import pandas as pd

### Automatizzazione della lettura

In [136]:
code = []
f = open('country_codes.txt')
for x in f:
    x = x.strip('\n')
    code.append(x)

In [169]:
# Inserire la data da considerare
day = 22
month = 12

In [167]:
dataframes = []
for i in code:
    file = "output_" + str(day)+"-" + str(month) + "-19/19." + str(day)+ "." + str(month) + "_" + str(i) + "_videos.csv"
    df = pd.read_csv(file)
    df['country_code'] = i
    dataframes.append(df)
YoutubeTrend = pd.concat(dataframes, axis = 0)
YoutubeTrend = YoutubeTrend.reset_index()

Conservo questo box sotto in caso di utilità

In [135]:
import os
import re
c = []
for filename in os.listdir(r'/home/marco/Scrivania/Università/Youtube/Youtube_trending_analysis/output_22-12-19'):
    m = re.search('_([A-Z]{2})_', filename)
    c.append(m.group(1))
c

['DE', 'IT', 'GB', 'MX', 'KR', 'JP', 'IN', 'US', 'RU', 'BR', 'CA', 'FR']

#### Video in trending più _vecchio_ pubblicato in assoluto

In [168]:
YoutubeTrend['publishedAt'] = pd.to_datetime(YoutubeTrend['publishedAt'])
YoutubeTrend['publishedAt'].sort_values()

159    2019-11-18 18:15:01+00:00
832    2019-11-20 03:59:48+00:00
2036   2019-11-22 13:05:32+00:00
1052   2019-11-22 13:05:32+00:00
350    2019-11-22 13:05:32+00:00
                  ...           
1210   2019-12-22 01:45:58+00:00
1192   2019-12-22 02:47:51+00:00
1184   2019-12-22 03:19:11+00:00
1177   2019-12-22 04:07:06+00:00
432    2019-12-22 04:30:22+00:00
Name: publishedAt, Length: 2322, dtype: datetime64[ns, UTC]

In [147]:
YoutubeTrend.loc[159][['title','publishedAt','view_count','likes','dislikes','comment_count','country_code']]

title            How to get the best fit and sound with your Po...
publishedAt                              2019-11-18 18:15:01+00:00
view_count                                                   78941
likes                                                          662
dislikes                                                       169
comment_count                                                    0
country_code                                                    US
Name: 159, dtype: object

#### Video in trending più _vecchio_ pubblicato relativo ai paesi

In [152]:
m = YoutubeTrend.groupby('country_code')['publishedAt'].idxmin()
#YoutubeTrend.loc[m]

#### Video con _visualizzazioni likes dislikes commenti_ maggiori

In [154]:
m = YoutubeTrend[['view_count','likes','dislikes','comment_count']].idxmax()
#YoutubeTrend.loc[m]

#### Video con _visualizzazioni likes dislikes commenti_ maggiori per paese

In [158]:
m = YoutubeTrend.groupby('country_code')['view_count'].idxmax()
#YoutubeTrend.loc[m]

#### Media di _visualizzazioni likes dislikes commenti_ del trending delle giornata

In [159]:
YoutubeTrend[['view_count','likes','dislikes','comment_count']].mean()

view_count       1.368667e+06
likes            6.640778e+04
dislikes         5.322524e+03
comment_count    5.680503e+03
dtype: float64

In [160]:
YoutubeTrend.groupby('country_code')[['view_count','likes','dislikes','comment_count']].mean()

,view_count,likes,dislikes,comment_count
country_code,,,,
BR,6.796129e+05,56745.560000,1076.990000,2922.710000
CA,2.151074e+06,106683.320000,2606.980000,9592.915000
DE,1.383706e+06,73405.085000,2362.435000,6435.355000
FR,1.225200e+06,62935.610000,1302.300000,4499.525000
GB,2.051334e+06,92747.640000,2210.995000,7733.765000
IN,1.933089e+06,65321.310559,3246.788820,4038.105590
IT,8.788939e+05,39041.700000,1055.465000,3442.470000
JP,1.163384e+06,32294.943590,41250.502564,6784.035897
KR,4.171489e+05,14967.054217,338.614458,1457.433735


#### _Categoria_ che riscuote maggior successo

le categorie sono indicate da numeri, il corrispettivo in testo è dato da dei file in json, per ora sto guardando quelli del tizio su kaggle però ci sarà un modo per trovarli anche direttamente tramite API. Questa cosa è molto figa perchè possiamo dire di utilizzare 2 tipi di dato, uno csv e l'altro json (anche se il json è semplice quindi magari meglio trovare altre fonti da unire a questi dati ahaha)

In [4]:
df.groupby('categoryId').count()['video_id']

categoryId
1      7
2      3
10    20
15     1
17    30
19     1
20    12
22    14
23    17
24    68
25     8
26     6
27     6
28     7
Name: video_id, dtype: int64

In [177]:
import json

In [187]:
with open(r'category_id/CA_category_id.json') as data:
    dat  = json.load(data)
dic={}
for elem in dat['items']:
    dic[elem['id']] = elem['snippet']['title']
cat = pd.DataFrame.from_dict(dic, orient = 'index')
cat.columns = ['Category']
cat

,Category
1,Film & Animation
2,Autos & Vehicles
10,Music
15,Pets & Animals
17,Sports
18,Short Movies
19,Travel & Events
20,Gaming
21,Videoblogging
22,People & Blogs


#### Popolazione dei singoli paesi (Fonte: UN)

In [101]:
pop = pd.read_csv('popUN.csv')

In [90]:
# Calcolo della popolazione dei singoli paesi
pop.loc[(pop['Age'] == 'Total') & (pop['Sex'] == 'Both Sexes') &(pop['Area'] == 'Total') ]

,Country or Area,Year,Area,Sex,Age,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Brazil,2010,Total,Both Sexes,Total,Census - de jure - complete tabulation,"Final figure, complete",2011.0,190755799.0,NaN
1107,Canada,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2019.0,34004889.0,"1,2"
1476,France,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2013.0,62765235.0,3
1477,France,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2012.0,62959391.0,"3,4"
1866,Germany,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2011.0,81802257.0,NaN
1867,Germany,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2011.0,81776929.5,NaN
2568,Italy,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2013.0,59277416.5,NaN
2569,Italy,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2013.0,59190143.0,NaN
3306,Japan,2010,Total,Both Sexes,Total,Census - de jure - complete tabulation,"Final figure, complete",2012.0,128057352.0,5
3307,Japan,2010,Total,Both Sexes,Total,Estimate - de jure,"Final figure, complete",2011.0,127450000.0,"5,6"
